In [1]:
import sys
import os
sys.path.insert(0, os.path.abspath('..'))
from os.path import expanduser

%load_ext autoreload
%autoreload 2

from dotenv import load_dotenv
load_dotenv()

from google.cloud import storage
from project_config import GCP_PROJECT_NAME, DATASET_JSON_PATH

gcp_client = storage.Client(project=GCP_PROJECT_NAME)

/home/ando/.conda/envs/rv_timm0412/lib/python3.9/site-packages/geopandas/_compat.py:124: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.3-CAPI-1.16.1). Conversions between both will be slow.
  warnings.warn(


### Configuration

In [2]:
from experiment_configs.configs import *
config = satmae_large_config

### Create Rastervision datasets

In [6]:
from torch.utils.data import ConcatDataset
import json
from utils.rastervision_pipeline import observation_to_scene, scene_to_training_ds, scene_to_validation_ds, warn_if_nan_in_raw_raster
from utils.data_management import observation_factory, characterize_dataset
import random

#set the seed
random.seed(13)

# get the current working directory
root_dir = os.getcwd()

# define the relative path to the dataset JSON file
json_rel_path = '../' + DATASET_JSON_PATH

# combine the root directory with the relative path
json_abs_path = os.path.join(root_dir, json_rel_path)

dataset_json = json.load(open(json_abs_path, 'r'))
all_observations = observation_factory(dataset_json)

#find the highest cluster id
max_cluster_id = max([observation['cluster_id'] for observation in dataset_json])


# Randomly split the data into training and validation
val_split = random.randint(0, max_cluster_id+1)

training_scenes = []
validation_scenes = []

for observation in all_observations:
    if observation.cluster_id == val_split: 
        validation_scenes.append(observation_to_scene(config, observation))
    else:
        training_scenes.append(observation_to_scene(config, observation))


training_datasets = [
    scene_to_training_ds(config, scene) for scene in training_scenes #random window sampling happens here
]
validation_datasets = [
    scene_to_validation_ds(config, scene) for scene in validation_scenes
]

train_dataset_merged = ConcatDataset(training_datasets)
val_dataset_merged = ConcatDataset(validation_datasets)

print('Validation split cluster_id:', val_split)
print ('Training dataset size: {:4d} images | Number of observations: {:}'.format(len(train_dataset_merged), len(training_scenes)))
print ('Testing dataset size: {:4d}  images | Number of observations: {:}'.format(len(val_dataset_merged), len(validation_scenes)))

mine_percentage_aoi = characterize_dataset(training_scenes, validation_scenes)

2023-11-30 18:20:37:rastervision.pipeline.file_system.utils: INFO - Using cached file /data/tmp/cache/http/storage.googleapis.com/sand_mining_median/labels/Kathajodi_Cuttack_85-85_20-44_median/s2/Kathajodi_Cuttack_85-85_20-44_2022-05-01_s2.tif.


2023-11-30 18:20:37:rastervision.pipeline.file_system.utils: INFO - Using cached file /data/tmp/cache/http/storage.googleapis.com/sand_mining_median/labels/Kathajodi_Cuttack_85-85_20-44_median/annotations/Kathajodi_Cuttack_85-85_20-44_2022-05-01_annotations.geojson.
2023-11-30 18:20:37:rastervision.pipeline.file_system.utils: INFO - Using cached file /data/tmp/cache/http/storage.googleapis.com/sand_mining_median/labels/Kathajodi_Cuttack_85-85_20-44_median/rivers/Kathajodi_Cuttack_85-85_20-44_rivers_1000m.geojson.
2023-11-30 18:20:37:rastervision.core.data.vector_source.geojson_vector_source: INFO - Ignoring CRS ({'type': 'name', 'properties': {'name': 'urn:ogc:def:crs:OGC:1.3:CRS84'}}) specified in https://storage.googleapis.com/sand_mining_median/labels/Kathajodi_Cuttack_85-85_20-44_median/rivers/Kathajodi_Cuttack_85-85_20-44_rivers_1000m.geojson and assuming EPSG:4326 instead.
2023-11-30 18:20:37:rastervision.pipeline.file_system.utils: INFO - Using cached file /data/tmp/cache/http/s

Validation split cluster_id: 4
Training dataset size: 3088 images | Number of observations: 72
Testing dataset size:  378  images | Number of observations: 8
Total dataset has 2.93%  mining area.
Training dataset has 3.05%  mining area.
Validation dataset has 2.15%  mining area.
Within AOIs, total dataset has 6.72%  mining area.
Outside AOIs, total dataset has 0.06%  mining area.

The median percentage of mine in an observation is 3.26%
The median number of mine pixels in an observation is 38059

The median number pixels in an observation is 1301008


## Update the loss weights

In [8]:
config.mine_class_loss_weight = (100 - mine_percentage_aoi) / mine_percentage_aoi
config

SupervisedFinetuningCofig(model_type=<ModelChoice.SatmaeLargeDoubleUpsampling: 'satmae-large-double-upsampling'>, optimizer=<OptimizerChoice.AdamW: 'adamw'>, tile_size=160, s2_channels=[1, 2, 3, 4, 5, 6, 7, 8, 10, 11], s2_normalization=<NormalizationS2Choice.ChannelWise: 'channelwise'>, loss_fn=<BackpropLossChoice.BCE: 'BCE'>, batch_size=128, learning_rate=0.001, datasets=<DatasetChoice.S2: 's2'>, mine_class_loss_weight=13.880675775871904, finetuning_strategy=<FinetuningStratagyChoice.LinearProbing: 'linear-probing'>, encoder_weights_path='/data/sand_mining/checkpoints/satmae_orig/pretrain-vit-large-e199.pth')

## Visualize the datasets

In [4]:
from utils.visualizing import visualize_dataset

print ('Training Dataset')
for ds in training_datasets:
    visualize_dataset(ds)

print("\n\n\n Test Dataset")

for ds in validation_datasets:
    visualize_dataset(ds)

Training Dataset


# Train

In [11]:
from models.model_factory import model_factory
from ml.optimizer_factory import optimizer_factory
from ml.learner import BinarySegmentationLearner

_, _, n_channels = training_datasets[0].scene.raster_source.shape
model = model_factory(
    config,
    n_channels=n_channels,
)

optimizer = optimizer_factory(config, model)

learner = BinarySegmentationLearner(
    config=config,
    model=model,
    optimizer=optimizer,
    train_ds=train_dataset_merged,  # for development and debugging, use training_datasets[0] or similar to speed up
    valid_ds=val_dataset_merged,  # for development and debugging, use training_datasets[1] or similar to speed up
    output_dir=expanduser("~/sandmining-watch/out/OUTPUT_DIR"),
)

learner.log_data_stats()

SatMae: Loading encoder weights from /data/sand_mining/checkpoints/satmae_orig/pretrain-vit-large-e199.pth
Position interpolate from 12x12 to 20x20
_IncompatibleKeys(missing_keys=['channel_cls_embed', 'head.weight', 'head.bias'], unexpected_keys=['mask_token', 'decoder_pos_embed', 'decoder_channel_embed', 'decoder_embed.weight', 'decoder_embed.bias', 'decoder_blocks.0.norm1.weight', 'decoder_blocks.0.norm1.bias', 'decoder_blocks.0.attn.qkv.weight', 'decoder_blocks.0.attn.qkv.bias', 'decoder_blocks.0.attn.proj.weight', 'decoder_blocks.0.attn.proj.bias', 'decoder_blocks.0.norm2.weight', 'decoder_blocks.0.norm2.bias', 'decoder_blocks.0.mlp.fc1.weight', 'decoder_blocks.0.mlp.fc1.bias', 'decoder_blocks.0.mlp.fc2.weight', 'decoder_blocks.0.mlp.fc2.bias', 'decoder_blocks.1.norm1.weight', 'decoder_blocks.1.norm1.bias', 'decoder_blocks.1.attn.qkv.weight', 'decoder_blocks.1.attn.qkv.bias', 'decoder_blocks.1.attn.proj.weight', 'decoder_blocks.1.attn.proj.bias', 'decoder_blocks.1.norm2.weight', 'd

2023-11-30 18:29:40:rastervision: INFO - train_ds: 3088 items
2023-11-30 18:29:40:rastervision: INFO - valid_ds: 378 items


#### Check GPU Activity

You can continuously monitor your GPU activity by using the command in the terminal


`watch -d -n 0.5 nvidia-smi`

In [12]:
# Run this cell if you want to log the run to W&B. You might need to authenticate to W&B.
learner.initialize_wandb_run()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: andoshah (sandmining-watch). Use `wandb login --relogin` to force relogin


wandb: logging graph, to disable use `wandb.watch(log_graph=False)`


In [7]:
learner.train(epochs=1)

2023-11-30 07:25:29:rastervision: INFO - epoch: 0


Training:   0%|          | 0/25 [00:00<?, ?it/s]

Validating:   0%|          | 0/3 [00:00<?, ?it/s]

2023-11-30 07:31:16:rastervision: INFO - metrics:
{'avg_f1': 0.902629554271698,
 'avg_precision': 0.971852719783783,
 'avg_recall': 0.8426119685173035,
 'epoch': 0,
 'other_f1': 0.9115577936172485,
 'other_precision': 0.9988391399383545,
 'other_recall': 0.8383044004440308,
 'sandmine_average_precision': tensor(0.6282, device='cuda:0'),
 'sandmine_f1': 0.28603678941726685,
 'sandmine_precision': 0.16772311925888062,
 'sandmine_recall': 0.9709680676460266,
 'train_bce_loss': 0.0035413133048022966,
 'train_dice_loss': 0.006418412213498446,
 'train_time': datetime.timedelta(seconds=304, microseconds=6428),
 'val_bce_loss': tensor(0.0033, device='cuda:0'),
 'val_dice_loss': tensor(0.0070, device='cuda:0'),
 'valid_time': datetime.timedelta(seconds=43, microseconds=549796)}
2023-11-30 07:31:18:rastervision: INFO - epoch: 1


Training:   0%|          | 0/25 [00:00<?, ?it/s]

...

Validating:   0%|          | 0/3 [00:00<?, ?it/s]

2023-11-30 09:12:58:rastervision: INFO - metrics:
{'avg_f1': 0.9307929873466492,
 'avg_precision': 0.972937285900116,
 'avg_recall': 0.8921481966972351,
 'epoch': 18,
 'other_f1': 0.9410833716392517,
 'other_precision': 0.9980452656745911,
 'other_recall': 0.8902723789215088,
 'sandmine_average_precision': tensor(0.6387, device='cuda:0'),
 'sandmine_f1': 0.36339759826660156,
 'sandmine_precision': 0.22477920353412628,
 'sandmine_recall': 0.9480436444282532,
 'train_bce_loss': 0.0031256558363919433,
 'train_dice_loss': 0.006405734027605601,
 'train_time': datetime.timedelta(seconds=294, microseconds=97215),
 'val_bce_loss': tensor(0.0026, device='cuda:0'),
 'val_dice_loss': tensor(0.0069, device='cuda:0'),
 'valid_time': datetime.timedelta(seconds=43, microseconds=449964)}
2023-11-30 09:13:00:rastervision: INFO - epoch: 19


Training:   0%|          | 0/25 [00:00<?, ?it/s]

Validating:   0%|          | 0/3 [00:00<?, ?it/s]

2023-11-30 09:18:37:rastervision: INFO - metrics:
{'avg_f1': 0.9428982138633728,
 'avg_precision': 0.9737053513526917,
 'avg_recall': 0.9139807820320129,
 'epoch': 19,
 'other_f1': 0.9535909295082092,
 'other_precision': 0.9974827766418457,
 'other_recall': 0.9133989810943604,
 'sandmine_average_precision': tensor(0.6285, device='cuda:0'),
 'sandmine_f1': 0.41283372044563293,
 'sandmine_precision': 0.2651945948600769,
 'sandmine_recall': 0.9313157796859741,
 'train_bce_loss': 0.0031257676337049416,
 'train_dice_loss': 0.006446888409747978,
 'train_time': datetime.timedelta(seconds=293, microseconds=668967),
 'val_bce_loss': tensor(0.0024, device='cuda:0'),
 'val_dice_loss': tensor(0.0068, device='cuda:0'),
 'valid_time': datetime.timedelta(seconds=43, microseconds=343775)}


## Evaluate 

Initialize evaluation_datasets and predictor.
evaluation_datasets and validation_datasets are based on identical scenes, but have different sliding window configurations.

In [13]:
from ml.learner import BinarySegmentationPredictor
from utils.rastervision_pipeline import scene_to_inference_ds

evaluation_datasets =  [
    scene_to_inference_ds(
        config, scene, full_image=True
    ) for scene in validation_scenes
]

predictor = BinarySegmentationPredictor(
    config,
    model,
)

## Alternatively: specify path to trained weights
# path_to_weights = expanduser("~/sandmining-watch/out/OUTPUT_DIR/SatMAE-L_w15.47_b128.pth")
# predictor = BinarySegmentationPredictor(
#     config,
#     model,
#     path_to_weights,
# )

Loading weights from /home/ando/sandmining-watch/out/OUTPUT_DIR/SatMAE-L_w15.47_b128.pth


In [13]:
from ml.eval_utils import evaluate_predicitions, make_wandb_segmentation_masks, make_wandb_predicted_probs_images
from utils.visualizing import raster_source_to_rgb

prediction_results_list = []

for ds in evaluation_datasets:
    predictions = predictor.predict_mine_probability_for_site(ds)

    rgb_img = raster_source_to_rgb(ds.scene.raster_source)
    prediction_results_list.append({
        "predictions": predictions,
        "ground_truth": ds.scene.label_source.get_label_arr(),
        "rgb_img": rgb_img,
        "name": ds.scene.id
    })

evaluation_results_dict = evaluate_predicitions(prediction_results_list)

Log results to Weights & Biases

In [10]:
import wandb

assert wandb.run is not None

# Add lists of W&B images to dict
evaluation_results_dict.update({
    'Segmenation masks': make_wandb_segmentation_masks(prediction_results_list),
    'Predicted probabilites': make_wandb_predicted_probs_images(prediction_results_list),
})

# Log to W&B
wandb.log(evaluation_results_dict)

In [11]:
wandb.finish()

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


eval/Betwa_Jalaun_79-49_25-84_2022-10-01/average_precision,▁
eval/Betwa_Jalaun_79-49_25-84_2022-10-01/f1_score,▁
eval/Betwa_Jalaun_79-49_25-84_2022-10-01/precision,▁
eval/Betwa_Jalaun_79-49_25-84_2022-10-01/recall,▁
eval/Betwa_Jalaun_79-49_25-84_2023-05-01/average_precision,▁
eval/Betwa_Jalaun_79-49_25-84_2023-05-01/f1_score,▁
eval/Betwa_Jalaun_79-49_25-84_2023-05-01/precision,▁
eval/Betwa_Jalaun_79-49_25-84_2023-05-01/recall,▁
eval/Betwa_Jalaun_79-79_25-89_2022-10-01/average_precision,▁
eval/Betwa_Jalaun_79-79_25-89_2022-10-01/f1_score,▁
eval/Betwa_Jalaun_79-79_25-89_2022-10-01/precision,▁
